# Generar Matriz de Distancias

### Instrucciones:

- **a. Origenes**  
  - Es una capa shapefile de polígonos con los campos que quiero calcular.  
  - Tiene que tener los nombres de los campos en una columna llamada `"idest"`.  
  - En general, esta capa es la capa de cultivos o una parte de ella.

- **b. Destinos**  
  - Es una capa shapefile de puntos con los destinos.  
  - Debe tener la columna `"Localidad"` con las localidades de los puertos, molinos o acopios de la capa.

In [1]:
import pandas as pd
import geopandas as gpd
import datetime as datetime
import os
from pathlib import Path
import time
import requests

In [2]:
try:
  espcul_gdf = gpd.read_file('./inputs/origenes.shp')

except Exception as e:
  espcul_gdf = gpd.read_file('./inputs/origenes.gpkg')

espcul_gdf = espcul_gdf.loc[espcul_gdf.campania == '25/26']

espcul_gdf = espcul_gdf[['zona', 'idest', 'geometry']]

print(espcul_gdf.shape)
espcul_gdf.head()

(1713, 3)


/Users/ju/Library/CloudStorage/OneDrive-ESPARTINAS.A/DocumentacionEspartina/INNOVACION/Desarrollos propios/comercial/distancias_destinos_localserver/.venv/lib/python3.11/site-packages/pyogrio/raw.py:198: RuntimeWarning: ./inputs/origenes.shp contains polygon(s) with rings with invalid winding order. Autocorrecting them, but that shapefile should be corrected using ogr2ogr for example.
  return ogr_read(


,zona,idest,geometry
0,LA PAMPA,Los Robles Lonquimay,"POLYGON ((-63.80458 -36.56039, -63.80444 -36.5..."
1,LA PAMPA,Atreuco,"POLYGON ((-63.83485 -37.0875, -63.83486 -37.08..."
2,LA PAMPA,Atreuco,"POLYGON ((-63.80902 -37.08386, -63.7928 -37.08..."
3,BA SDE N,Banderitas,"MULTIPOLYGON (((-58.32488 -36.15052, -58.32478..."
4,BA COS N,Las Overas,"POLYGON ((-58.4547 -38.49917, -58.46121 -38.50..."


In [3]:
zonas_list = espcul_gdf.zona.unique()
zonas_list

array(['LA PAMPA', 'BA SDE N', 'BA COS N', 'BA OESTE DX', 'BA OESTE',
       'BA SDE S', 'SALTA', 'BA NORTE', 'CBA SUR', 'LA PAMPA NORTE',
       'ENTRE RIOS', 'SANTA FE N', 'SANTIAGO DEL ESTERO', 'CHACO',
       'SAN LUIS'], dtype=object)

In [4]:
# espcul_gdf = espcul_gdf.loc[espcul_gdf.zona.isin(['LA PAMPA'])]

In [5]:
espcul_gdf = gpd.GeoDataFrame(espcul_gdf, geometry='geometry').reset_index()
espcul_gdf['geometry'] = espcul_gdf.buffer(0.0001)
espcul_dissolve = espcul_gdf.dissolve(by='idest').reset_index()

print(espcul_dissolve.shape)
espcul_dissolve.head()

/var/folders/xt/0fzdrsd52bj4z2cqvy77d5w00000gn/T/ipykernel_33889/13057875.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  espcul_gdf['geometry'] = espcul_gdf.buffer(0.0001)
/Users/ju/Library/CloudStorage/OneDrive-ESPARTINAS.A/DocumentacionEspartina/INNOVACION/Desarrollos propios/comercial/distancias_destinos_localserver/.venv/lib/python3.11/site-packages/shapely/constructive.py:246: RuntimeWarning: invalid value encountered in buffer
  return lib.buffer(


(210, 4)


,idest,geometry,index,zona
0,4 Marias,"MULTIPOLYGON (((-61.05521 -26.30975, -61.0552 ...",723,CHACO
1,Agrotecnico,"MULTIPOLYGON (((-64.26534 -35.16614, -64.26534...",599,LA PAMPA NORTE
2,Aitona,"MULTIPOLYGON (((-57.60189 -37.76202, -57.60189...",28,BA COS N
3,Alto Verde,"POLYGON ((-64.09036 -34.82056, -64.09037 -34.8...",1560,CBA SUR
4,Angel de la Guarda,"MULTIPOLYGON (((-61.02682 -26.10023, -61.02683...",960,CHACO


In [6]:
espcul_dissolve['x']= espcul_dissolve['geometry'].centroid.x
espcul_dissolve['y']= espcul_dissolve['geometry'].centroid.y
espcul_dissolve['centres']= espcul_dissolve['geometry'].centroid
espcul_centroids = espcul_dissolve.loc[:,['idest','x','y','centres']].copy()
espcul_centroids = espcul_centroids.rename(columns = {'centres': 'geometry'})
espcul_centroids = espcul_centroids.to_crs('EPSG:4326')
espcul_centroids.head()

/var/folders/xt/0fzdrsd52bj4z2cqvy77d5w00000gn/T/ipykernel_33889/1692939824.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  espcul_dissolve['x']= espcul_dissolve['geometry'].centroid.x
/var/folders/xt/0fzdrsd52bj4z2cqvy77d5w00000gn/T/ipykernel_33889/1692939824.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  espcul_dissolve['y']= espcul_dissolve['geometry'].centroid.y
/var/folders/xt/0fzdrsd52bj4z2cqvy77d5w00000gn/T/ipykernel_33889/1692939824.py:3: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  espcul_dissolve['centres']= espcul_dissolve['geometry'].centroid


,idest,x,y,geometry
0,4 Marias,-61.042354,-26.306969,POINT (-61.04235 -26.30697)
1,Agrotecnico,-64.280203,-35.148725,POINT (-64.2802 -35.14872)
2,Aitona,-57.588661,-37.755373,POINT (-57.58866 -37.75537)
3,Alto Verde,-64.090822,-34.806184,POINT (-64.09082 -34.80618)
4,Angel de la Guarda,-61.020956,-26.084990,POINT (-61.02096 -26.08499)


In [7]:
espcul_centroids.to_file(driver = "GPKG",filename='./inputs/centoids.gpkg', encoding='utf-8', index=False)

In [8]:
espcul_coord = {}

for i in range(0, len(espcul_centroids)):
    origin = espcul_centroids['idest'][i]
    x = espcul_centroids['geometry'][i].x
    y = espcul_centroids['geometry'][i].y
    coords = (y, x)
    espcul_coord[origin]=coords
# espcul_coord

In [9]:
try:
  destinos_gdf = gpd.read_file('./inputs/destinos.shp')

except Exception as e:
  destinos_gdf = gpd.read_file('./inputs/destinos.gpkg')


print(destinos_gdf.shape)
destinos_gdf.head()

destinos_coord = {}

for i in range(len(destinos_gdf)):
    destino = destinos_gdf['Localidad'][i]
    punto = destinos_gdf['geometry'][i].centroid
    coords = (punto.y, punto.x)
    destinos_coord[destino] = coords

list(destinos_coord.items())[:4]

(1, 4)


[('terminal_las_palmas', (-34.01746304329869, -59.1214643244392))]

The centroids may be away from actual street network. Use local OSRM server to snap coordinates to the nearest road.

In [10]:
def snap_to_road(lat, lon, port=5001):
    """
    Snaps a coordinate to the nearest road using local OSRM server.
    
    Parameters:
    - lat: latitude
    - lon: longitude
    - port: local port where OSRM server is running (default is 5001)
    
    Returns:
    - tuple (lat, lon) snapped to nearest road
    """
    url = f"http://localhost:{port}/nearest/v1/driving/{lon},{lat}"
    try:
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            if data['code'] == 'Ok':
                # Returns [lon, lat]
                snapped = data['waypoints'][0]['location']
                return (snapped[1], snapped[0])  # Return as (lat, lon)
        return (lat, lon)  # Return original if fails
    except Exception as e:
        print(f"Error connecting to OSRM server: {e}")
        return (lat, lon)

In [11]:
# Snap all centroids to road network using local OSRM
espcul_coord_snapped = {}
for name, coords in espcul_coord.items():
    espcul_coord_snapped[name] = snap_to_road(coords[0], coords[1], port=5001)

print(f"Snapped {len(espcul_coord_snapped)} coordinates to road network")

Snapped 210 coordinates to road network


In [12]:
# Display snapped coordinates
espcul_coord_snapped

{'4 Marias': (-26.302134, -61.049085),
 'Agrotecnico': (-35.148716, -64.265026),
 'Aitona': (-37.748101, -57.586615),
 'Alto Verde': (-34.806324, -64.096943),
 'Angel de la Guarda': (-26.085251, -61.00246),
 'Aprea': (-38.351407, -58.623965),
 'Atreuco': (-37.077776, -63.815562),
 'Banderitas': (-36.158565, -58.299006),
 'Bella Vista': (-37.232504, -57.558987),
 'Bollo': (-33.790935, -64.297905),
 'Campanella': (-38.721511, -59.664096),
 'Campo Rivero': (-38.499165, -58.833545),
 'Cardal del Monte': (-35.635772, -58.648117),
 'Carrizales': (-36.151855, -58.554435),
 'Cerro Destino II': (-37.586779, -59.272869),
 'Chapaico': (-36.684429, -62.038031),
 'Colonia Fourcade': (-35.88688, -58.45895),
 'Corvalan': (-36.19986, -58.168427),
 'Cupalen': (-37.245811, -58.488299),
 'Cupalen II': (-37.242393, -58.489683),
 'Don Eduardo': (-35.617214, -58.513446),
 'Don Francisco': (-35.794731, -62.751973),
 'Don Gustavo': (-37.350229, -57.29642),
 'Don Justo': (-36.40132, -61.919655),
 'Don Roberto'

In [13]:
def get_driving_distance(source_coordinates, dest_coordinates, port=5001):
    """
    Returns the driving distance in kilometers between two coordinates using a local OSRM server.

    Parameters:
    - source_coordinates: tuple (lat, lon)
    - dest_coordinates: tuple (lat, lon)
    - port: local port where OSRM server is running (default is 5000)

    Returns:
    - distance in kilometers (float)
    - returns -9999 if request fails
    """
    base_url = f"http://localhost:{port}/route/v1/driving"
    # OSRM expects lon,lat
    src_lon, src_lat = source_coordinates[1], source_coordinates[0]
    dst_lon, dst_lat = dest_coordinates[1], dest_coordinates[0]

    url = f"{base_url}/{src_lon},{src_lat};{dst_lon},{dst_lat}?overview=false"

    try:
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            distance = data['routes'][0]['distance']  # in meters
            return distance / 1000  # return in kilometers
        else:
            print(f"Request failed. Status code: {response.status_code}")
            return -9999
    except Exception as e:
        print(f"Error connecting to OSRM server: {e}")
        return -9999

In [14]:
rows = []

for k1, v1 in espcul_coord_snapped.items():
    for k2, v2 in destinos_coord.items():
        origin = v1
        destino = v2
        distance = get_driving_distance(origin, destino, port=5001)
        
        rows.append({
            'campo': k1,
            'campo_x': v1[1],
            'campo_y': v1[0],
            'destino': k2,
            'destino_x': v2[1],
            'destino_y': v2[0],
            'distancia': int(distance)
        })

df = pd.DataFrame(rows)

# df.head()

In [15]:
resultados = pd.merge(df, espcul_dissolve[['idest', 'zona']], left_on='campo', right_on='idest', how='left').drop(columns='idest')
resultados = resultados[['zona', 'campo', 'campo_x', 'campo_y', 'destino', 'destino_x', 'destino_y', 'distancia']]
resultados.head()

,zona,campo,campo_x,campo_y,destino,destino_x,destino_y,distancia
0,CHACO,4 Marias,-61.049085,-26.302134,terminal_las_palmas,-59.121464,-34.017463,1144
1,LA PAMPA NORTE,Agrotecnico,-64.265026,-35.148716,terminal_las_palmas,-59.121464,-34.017463,557
2,BA COS N,Aitona,-57.586615,-37.748101,terminal_las_palmas,-59.121464,-34.017463,496
3,CBA SUR,Alto Verde,-64.096943,-34.806324,terminal_las_palmas,-59.121464,-34.017463,559
4,CHACO,Angel de la Guarda,-61.002460,-26.085251,terminal_las_palmas,-59.121464,-34.017463,1170


In [16]:
print(f'df original: {df.shape}')
print(f'df zonas: {resultados.shape}')

df original: (210, 7)
df zonas: (210, 8)


In [18]:
path = "./outputs"

if not os.path.exists(path):
    os.makedirs(path)

# df.to_csv('matrix.csv', index=False)
resultados.to_excel('./outputs/matriz_de_distancias.xlsx', index=False)